# MPPI алгоритм для балансировки перевернутого маятника

Реализация алгоритма стохастического оптимального управления Model Predictive Path Integral с использованием JAX, Python + Pybind11 + C++ и Python + Numpy. Алгоритм находит решение для задачи балансировки перевернутого маятника.

## Импорт библиотек и настройка окружения

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import time

In [ ]:
project_root = Path.cwd().parent
src_path = project_root / 'src'
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

In [ ]:
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 100
plt.style.use('seaborn-v0_8-darkgrid')

In [ ]:
from controller import PendulumConfig, MPPIConfig
from controller.mppi_controller import MPPIController
from visualization import PendulumAnimator, Plotter
from utils import ResultsCollector, MetricsCalculator

In [ ]:
try:
    from mppi.numpy.mppi_numpy import MPPINumpy
    NUMPY_AVAILABLE = True
except ImportError:
    NUMPY_AVAILABLE = False
    print("NumPy реализация не доступна")

try:
    from mppi.jax.mppi_jax import MPPIJax
    JAX_AVAILABLE = True
except ImportError:
    JAX_AVAILABLE = False
    print("JAX реализация не доступна")

try:
    from mppi.cpp.mppi_cpp import MPPICpp
    CPP_AVAILABLE = True
except ImportError:
    CPP_AVAILABLE = False
    print("C++ реализация не доступна")

C++ реализация не доступна


##  Описание алгоритма MPPI

Немного подробней можно прочитать в `README.md`

 ### Что такое задача перевернутого маятника?

 Есть тележка, которая может двигаться влево-вправо, и закрепленный на ней маятник, который нужно удерживать в вертикальном положении. Система неустойчива, потому что маятник падает при малейшем отклонении.

### Уравнения движения

$$
\ddot{x} = \frac{F + m\sin\theta (l\dot{\theta}^2 + g\cos\theta)}{M + m\sin^2\theta}
$$

$$
\ddot{\theta} = \frac{-F\cos\theta - m l \dot{\theta}^2 \cos\theta \sin\theta - (M+m)\sin\theta}{l (M + m\sin^2\theta)}
$$

### Как работает MPPI?

1. **Получаем текущее состояние** системы (положение, скорость тележки, угол, скорость маятника)
2. **Генерируем K случайных траекторий** управления на T шагов вперед
3. **Просчитываем каждую траекторию** через модель динамики
4. **Вычисляем стоимость** каждой траектории
5. **Взвешиваем траектории** по формуле:
   $$w_i = \exp\left( -\frac{1}{\lambda} (s_i - \min_j s_j) \right)$$
6. **Усредняем траектории** с учетом весов
7. **Применяем первое управление** из оптимальной последовательности
8. **Повторяем** весь процесс

## Панель управления параметрами

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

### Параметры системы (маятник + тележка)

In [ ]:
system_widgets = widgets.VBox([
    widgets.Label("ПАРАМЕТРЫ СИСТЕМЫ", style={'font_weight': 'bold', 'font_size': '14px'}),

    widgets.FloatSlider(
        value=1.0,
        min=0.5,
        max=5.0,
        step=0.1,
        description='Масса тележки (кг):',
        continuous_update=False,
        style={'description_width': 'initial'}
    ),

    widgets.FloatSlider(
        value=0.1,
        min=0.05,
        max=2.0,
        step=0.05,
        description='Масса маятника (кг):',
        continuous_update=False,
        style={'description_width': 'initial'}
    ),

    widgets.FloatSlider(
        value=1.0,
        min=0.5,
        max=2.0,
        step=0.1,
        description='Длина маятника (м):',
        continuous_update=False,
        style={'description_width': 'initial'}
    ),

    widgets.FloatSlider(
        value=10.0,
        min=-30,
        max=30,
        step=1,
        description='Начальный угол (°):',
        continuous_update=False,
        style={'description_width': 'initial'}
    ),

    widgets.FloatSlider(
        value=0.0,
        min=-2.0,
        max=2.0,
        step=0.1,
        description='Начальная позиция (м):',
        continuous_update=False,
        style={'description_width': 'initial'}
    ),
])

display(system_widgets)

### Параметры алгоритма MPPI

In [ ]:
algorithm_widgets = widgets.VBox([
    widgets.Label("ПАРАМЕТРЫ АЛГОРИТМА MPPI", style={'font_weight': 'bold', 'font_size': '14px'}),

    widgets.IntSlider(
        value=1000,
        min=100,
        max=10000,
        step=100,
        description='Количество траекторий (K):',
        continuous_update=False,
        style={'description_width': 'initial'}
    ),

    widgets.IntSlider(
        value=30,
        min=10,
        max=100,
        step=5,
        description='Горизонт планирования (T):',
        continuous_update=False,
        style={'description_width': 'initial'}
    ),

    widgets.FloatSlider(
        value=1.0,
        min=0.1,
        max=10.0,
        step=0.1,
        description='Параметр λ:',
        continuous_update=False,
        style={'description_width': 'initial'}
    ),

    widgets.FloatSlider(
        value=0.1,
        min=0.01,
        max=1.0,
        step=0.01,
        description='Штраф за силу (R):',
        continuous_update=False,
        style={'description_width': 'initial'}
    )
])

display(algorithm_widgets)

СОВЕТЫ ПО НАСТРОЙКЕ ПАРАМЕТРОВ:

1. **Количество траекторий (K)**:
   - 100-500: Быстрее выполняется алгоритм, но менее точно
   - 500-2000: Что-то среднее
   - 2000-10000: Точнее, но медленнее

2. **Горизонт планирования (T)**:
   - 10-20: Быстрая реакция
   - 20-50: Что-то среднее
   - 50-100: Более долгосрочное планирование

3. **Параметр λ**:
   - 0.1-1.0: Более агрессивное управление
   - 1.0-5.0: Что-то среднее
   - 5.0-10.0: Более плавное управление

4. **Вес управления (R)**:
   - 0.01-0.1: Сильное управление, может быть резким
   - 0.1-0.5: Что-то среднее
   - 0.5-1.0: Плавное управление, экономия энергии

### Выбор реализации

In [ ]:
implementation_selector = widgets.RadioButtons(
    options=[
        ('Python + NumPy', 'numpy'),
        ('JAX (GPU ускорение)', 'jax'),
        ('C++ (Pybind11)', 'cpp'),
        ('Сравнить все', 'compare')
    ],
    value='numpy',
    description='Реализация:',
    disabled=False,
    style={'description_width': 'initial'}
)

display(implementation_selector)

RadioButtons(description='Реализация:', options=(('Python + NumPy', 'numpy'), ('JAX (GPU ускорение)', 'jax'), …

### Кнопки управления


In [ ]:
button_run = widgets.Button(
    description='Запустить',
    button_style='success',
    tooltip='Запустить симуляцию с выбранными параметрами',
    icon='play'
)

button_stop = widgets.Button(
    description='Остановить',
    button_style='danger',
    tooltip='Остановить текущую симуляцию',
    icon='stop',
    disabled=True
)

button_compare = widgets.Button(
    description='Сравнить все',
    button_style='info',
    tooltip='Запустить все реализации и сравнить производительность',
    icon='bar-chart'
)

button_container = widgets.HBox([button_run, button_stop, button_compare])
display(button_container)

In [ ]:
output_tabs = widgets.Tab()

animation_output = widgets.Output()
plots_output = widgets.Output()
metrics_output = widgets.Output()
comparison_output = widgets.Output()

output_tabs.children = [animation_output, plots_output, metrics_output, comparison_output]
output_tabs.set_title(0, 'Анимация')
output_tabs.set_title(1, 'Графики')
output_tabs.set_title(2, 'Метрики')
output_tabs.set_title(3, 'Сравнение')

display(output_tabs)

 ## Функции для управления симуляцией

In [ ]:
current_controller = None
is_running = False
results_collector = ResultsCollector()
simulation_results = {}

In [ ]:
def create_pendulum_config():
    """Создает конфигурацию системы из виджетов"""
    # Получаем значения из виджетов
    children = system_widgets.children
    
    return PendulumConfig(
        m_cart=children[1].value,      # Масса тележки
        m_pole=children[2].value,      # Масса маятника
        l=children[3].value,           # Длина маятника
        theta0=np.radians(children[4].value),  # Начальный угол (в радианах)
        x0=children[5].value,          # Начальная позиция
        g=9.81,                        # Ускорение свободного падения
        dt=0.02                        # Шаг симуляции
    )

In [ ]:
def create_mppi_config():
    """Создает конфигурацию MPPI из виджетов"""
    children = algorithm_widgets.children
    
    return MPPIConfig(
        K=children[1].value,           # Количество траекторий
        T=children[2].value,           # Горизонт планирования
        lambda_=children[3].value,     # Параметр λ
        sigma=1.0,                     # Стандартное отклонение шума
        R=children[4].value,           # Вес управления
        Q=[1.0, 10.0, 0.1, 0.1]       # Веса для состояния
    )

## Визуализация

In [ ]:
def visualize_results(result, implementation):
    """Визуализирует результаты симуляции"""
    
    # Получаем данные из результата
    history = result['history']
    metrics = result['metrics']
    
    states = np.array(history['states'])
    controls = np.array(history['controls'])
    costs = np.array(history['costs'])
    
    # Создаем анимацию
    with animation_output:
        clear_output(wait=True)
        print("Создание анимации...")
        
        try:
            animator = PendulumAnimator(
                history_states=states.tolist(),
                history_controls=controls.tolist(),
                pendulum_length=create_pendulum_config().l
            )
            
            animation = animator.create_animation(interval=50)
            
            from IPython.display import HTML
            display(HTML(animation.to_jshtml()))
            
        except Exception as e:
            print(f"Не удалось создать анимацию: {e}")
            print("Продолжаем с графиками...")
    
    # Создаем графики
    with plots_output:
        clear_output(wait=True)
        print(" Построение графиков...")
        
        # Создаем дашборд с результатами
        fig = Plotter.plot_simulation_results(history)
        plt.suptitle(f'Результаты симуляции - Реализация: {implementation}', fontsize=16)
        plt.show()
    
    #Показываем метрики
    with metrics_output:
        clear_output(wait=True)
        print("Метрики производительности:")
        print("="*50)
        
        for metric_name, metric_value in metrics.items():
            if isinstance(metric_value, float):
                print(f"{metric_name:30}: {metric_value:.4f}")
            else:
                print(f"{metric_name:30}: {metric_value}")
        
        print("\nВизуализация ключевых метрик:")
        
        # Выбираем ключевые метрики для визуализации
        key_metrics = {
            'avg_angle': 'Средний угол (°)',
            'max_angle': 'Макс. угол (°)',
            'avg_force': 'Средняя сила (Н)',
            'execution_time': 'Время выполнения (с)'
        }
        
        fig, ax = plt.subplots(figsize=(10, 6))
        
        metric_names = []
        metric_values = []
        
        for key, display_name in key_metrics.items():
            if key in metrics:
                metric_names.append(display_name)
                value = metrics[key]
                if 'angle' in key:
                    value = np.degrees(value)
                metric_values.append(value)
        
        if metric_values:
            colors = plt.cm.Set3(np.linspace(0, 1, len(metric_values)))
            bars = ax.bar(metric_names, metric_values, color=colors, alpha=0.8)

            for bar, value in zip(bars, metric_values):
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height + max(metric_values)*0.02,
                       f'{value:.2f}', ha='center', va='bottom')
            
            ax.set_ylabel('Значение')
            ax.set_title(f'Ключевые метрики - {implementation}')
            ax.grid(True, alpha=0.3, axis='y')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()


## Запуск симуляции

In [ ]:
def run_simulation(implementation):
    """Запускает симуляцию с выбранной реализацией"""
    global current_controller, is_running, simulation_results
    
    # Создаем конфигурации
    pendulum_config = create_pendulum_config()
    mppi_config = create_mppi_config()
    
    # Проверяем доступность реализации
    if implementation == 'numpy' and not NUMPY_AVAILABLE:
        print("NumPy реализация не доступна")
        return
    elif implementation == 'jax' and not JAX_AVAILABLE:
        print("JAX реализация не доступна")
        return
    elif implementation == 'cpp' and not CPP_AVAILABLE:
        print("C++ реализация не доступна")
        return
    
    try:
        # Создаем контроллер
        controller = MPPIController(pendulum_config, mppi_config, implementation=implementation)
        current_controller = controller
        is_running = True
        button_stop.disabled = False
        
        print(f"Запуск симуляции с параметрами:")
        print(f"Система: тележка={pendulum_config.m_cart}кг, маятник={pendulum_config.m_pole}кг")
        print(f"MPPI: K={mppi_config.K}, T={mppi_config.T}, λ={mppi_config.lambda_}")
        print(f"Реализация: {implementation}")
        
        num_steps = 500
         # Запускаем симуляцию
        with plots_output:
            clear_output(wait=True)
            print("Идет симуляция...")
            
           
            result = results_collector.run_simulation(controller, num_steps=num_steps, 
                                                     experiment_name=f"{implementation}_{int(time.time())}")
        
        print("Симуляция завершена!")
        
        # Сохраняем результаты
        simulation_results[implementation] = result
        
        # Визуализируем результаты
        visualize_results(result, implementation)
        
    except Exception as e:
        print(f"Ошибка при запуске симуляции: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
def run_all_implementations():
    """Запускает все реализации и сравнивает их"""
    global simulation_results
    
    implementations = []
    if NUMPY_AVAILABLE:
        implementations.append('numpy')
    if JAX_AVAILABLE:
        implementations.append('jax')
    if CPP_AVAILABLE:
        implementations.append('cpp')
    
    if len(implementations) < 2:
        print("Для сравнения нужно хотя бы 2 доступные реализации")
        return
    
    simulation_results = {}
    
    with comparison_output:
        clear_output(wait=True)
        print("Запуск сравнения всех реализаций...")
        
        for impl in implementations:
            print(f"\n Тестирование реализации: {impl}")
            
            # Создаем конфигурации
            pendulum_config = create_pendulum_config()
            mppi_config = create_mppi_config()
            
            # Создаем контроллер
            controller = MPPIController(pendulum_config, mppi_config, implementation=impl)
            
            # Запускаем симуляцию на 100 шагов для сравнения
            result = results_collector.run_simulation(
                controller, 
                num_steps=100,
                experiment_name=f"compare_{impl}_{int(time.time())}"
            )
            
            simulation_results[impl] = result
            metrics = result['metrics']
            
            print(f"Время выполнения: {metrics.get('execution_time', 0):.3f} с")
            print(f"Средний угол: {np.degrees(metrics.get('avg_angle', 0)):.1f}°")
            print(f"Макс. угол: {np.degrees(metrics.get('max_angle', 0)):.1f}°")
        
        # Строим график сравнения
        print("\nВИЗУАЛИЗАЦИЯ СРАВНЕНИЯ\n" + "="*50)
        
        # Получаем метрики для сравнения
        comparison_metrics = {}
        for impl, result in simulation_results.items():
            comparison_metrics[impl] = result['metrics']
        
        # График сравнения производительности
        fig = Plotter.plot_performance_comparison({
            impl: metrics.get('execution_time', 0) 
            for impl, metrics in comparison_metrics.items()
        })
        plt.show()
        
        # Детальное сравнение
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        
        # Время выполнения
        impl_names = list(comparison_metrics.keys())
        exec_times = [comparison_metrics[impl].get('execution_time', 0) for impl in impl_names]
        
        axes[0, 0].bar(impl_names, exec_times, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
        axes[0, 0].set_title('Время выполнения (с)')
        axes[0, 0].set_ylabel('Секунды')
        axes[0, 0].grid(True, alpha=0.3)
        
        for i, (impl, time_val) in enumerate(zip(impl_names, exec_times)):
            axes[0, 0].text(i, time_val + max(exec_times)*0.02, f'{time_val:.3f}', 
                           ha='center', va='bottom')
        
        # Средний угол
        avg_angles = [np.degrees(comparison_metrics[impl].get('avg_angle', 0)) for impl in impl_names]
        
        axes[0, 1].bar(impl_names, avg_angles, color=['#FFE66D', '#FF6B6B', '#4ECDC4'])
        axes[0, 1].set_title('Средний угол отклонения (°)')
        axes[0, 1].set_ylabel('Градусы')
        axes[0, 1].grid(True, alpha=0.3)
        
        for i, (impl, angle) in enumerate(zip(impl_names, avg_angles)):
            axes[0, 1].text(i, angle + max(avg_angles)*0.02, f'{angle:.1f}', 
                           ha='center', va='bottom')
        
        # Максимальный угол
        max_angles = [np.degrees(comparison_metrics[impl].get('max_angle', 0)) for impl in impl_names]
        
        axes[1, 0].bar(impl_names, max_angles, color=['#95E1D3', '#F38181', '#FCE38A'])
        axes[1, 0].set_title('Максимальный угол отклонения (°)')
        axes[1, 0].set_ylabel('Градусы')
        axes[1, 0].grid(True, alpha=0.3)
        
        for i, (impl, angle) in enumerate(zip(impl_names, max_angles)):
            axes[1, 0].text(i, angle + max(max_angles)*0.02, f'{angle:.1f}', 
                           ha='center', va='bottom')
        
        # Средняя сила
        avg_forces = [comparison_metrics[impl].get('avg_force', 0) for impl in impl_names]
        
        axes[1, 1].bar(impl_names, avg_forces, color=['#A8E6CF', '#DCEDC1', '#FFD3B6'])
        axes[1, 1].set_title('Средняя управляющая сила (Н)')
        axes[1, 1].set_ylabel('Ньютоны')
        axes[1, 1].grid(True, alpha=0.3)
        
        for i, (impl, force) in enumerate(zip(impl_names, avg_forces)):
            axes[1, 1].text(i, force + max(avg_forces)*0.02, f'{force:.2f}', 
                           ha='center', va='bottom')
        
        plt.suptitle('Детальное сравнение реализаций MPPI', fontsize=16)
        plt.tight_layout()
        plt.show()
        
        print("ИТОГИ СРАВНЕНИЯ")
        print("-" * 50)
        
        # Находим лучшие реализации
        if exec_times:
            best_time_idx = np.argmin(exec_times)
            best_time_impl = impl_names[best_time_idx]
            print(f"Самая быстрая: {best_time_impl} ({exec_times[best_time_idx]:.3f} с)")
        
        if avg_angles:
            best_angle_idx = np.argmin(avg_angles)
            best_angle_impl = impl_names[best_angle_idx]
            print(f"Самая стабильная: {best_angle_impl} (ср. угол: {avg_angles[best_angle_idx]:.1f}°)")
        
        if avg_forces:
            best_force_idx = np.argmin(avg_forces)
            best_force_impl = impl_names[best_force_idx]
            print(f"🔹 Самая экономичная: {best_force_impl} (ср. сила: {avg_forces[best_force_idx]:.2f} Н)")


In [ ]:
def on_run_button_clicked(b):
    """Обработчик клика по кнопке 'Запустить симуляцию'"""
    implementation = implementation_selector.value
    
    if implementation == 'compare':
        run_all_implementations()
    else:
        run_simulation(implementation)

def on_stop_button_clicked(b):
    """Обработчик клика по кнопке 'Остановить'"""
    global is_running
    is_running = False
    button_stop.disabled = True
    print("Симуляция остановлена")

def on_compare_button_clicked(b):
    """Обработчик клика по кнопке 'Сравнить все'"""
    run_all_implementations()

# Привязываем обработчики к кнопкам
button_run.on_click(on_run_button_clicked)
button_stop.on_click(on_stop_button_clicked)
button_compare.on_click(on_compare_button_clicked)

## Сохранение и загрузка результатов

In [ ]:
def save_experiment():
    """Сохраняет текущий эксперимент"""
    if results_collector.results:
        filename = f"experiment_{int(time.time())}.pkl"
        results_collector.save_results(filename)
        print(f"Эксперимент сохранен в {filename}")
    else:
        print("Нет данных для сохранения")

def load_experiment():
    """Загружает эксперимент из файла"""
    import pickle
    from pathlib import Path
    
    files = list(Path('.').glob('*.pkl'))
    if files:
        latest = max(files, key=lambda x: x.stat().st_mtime)
        results_collector.load_results(latest.name)
        print(f"Эксперимент загружен из {latest.name}")
        print(f"Количество записей: {len(results_collector.results)}")
    else:
        print("Нет сохраненных экспериментов")

In [ ]:
save_button = widgets.Button(
    description='Сохранить',
    button_style='primary',
    icon='save'
)

load_button = widgets.Button(
    description='Загрузить',
    button_style='primary',
    icon='folder'
)

def on_save_click(b):
    save_experiment()

def on_load_click(b):
    load_experiment()

In [ ]:
save_button.on_click(on_save_click)
load_button.on_click(on_load_click)

display(widgets.HBox([save_button, load_button]))